In [14]:
%%sh
# MC delete this cell at the end, as it is just for interfacing with the underlying linux environment.
ls ./report

Montagud2022_Prostate_Cancer.bnd
Montagud2022_Prostate_Cancer.cfg
Montagud2022_Prostate_Cancer.zginml
elife-72626-v2.pdf
supplementary_files


# Introduction to data analysis for natural and social sciences
This notebook contitutes the first part of the exam.

Here the steps of article "Patient-specific Boolean models of signalling networks guide personalised treatments" are retraced and the results reproduced.

## Imports and global settings

In [17]:
import pandas as pd

In [18]:
PATH_REPORT = "report"

# Prostate Boolean model construction
The Boolean model is constructed starting from biological information available in literature.